# NN Project, topic #4: Language identification from a text corpus

Students: Shahzaib MUHAMMAD, Adriana NICOARA, Scott TANKARD

## Our group's topic

From: https://arche.univ-lorraine.fr/mod/page/view.php?id=1310565

Project 4: Language identification from a text corpus

Students: Scott TANKARD, Adriana NICOARA, Shahzaib MUHAMMAD

Task: Text classification

Model: Feedforward neural network

Dataset: https://www.kaggle.com/zarajamshaid/language-identification-datasst

Hint: As a preprocessing step, you should transforms the sentences into trigrams at the character level (more details at: https://towardsdatascience.com/deep-neural-network-language-identification-ae1c158f6a7d)

## General project instructions

From: https://arche.univ-lorraine.fr/mod/page/view.php?id=1301340

General project instructions

Project structure

Each project follows the same overall structure:

    Download the data and preprocess it as required for the given task (if needed).
    Write a Dataset class for creating the train and test datasets (and corresponding dataloaders).
    Define the neural network model.
    Define the hyperparameters to create an instance of the model (e.g., hidden space size, number of convolution kernels...) as well as the parameters required to train neural network (e.g., learning rate).
    Write the training loop for training the model.
    Evaluate the model on the test data. In this part, it is expected to choose an appropriate evaluation metric based on your task. For instance, for classification task, accuracy should be computed (but you can also search for 'precision' and 'recall').
    Save the trained model parameters, and the obtained results if needed.


Deliverables

The project should be written using Pytorch (and not Keras/Tensorflow or any other python deep learning framework). You're expected to send a zip file containing:

    A python file implementing steps 1 to 7. This can be either a jupyter notebook or a .py script. A single file is preferred, but you can use several files if that's your style, as long as your main executable script is clearly indicated (for instance it's named 'main.py').
    The obtained results as extra files (e.g., the trained model parameters, a figure with the training loss / validation metric over epochs, a graph comparing different architectures if you want to play arround with it, etc.)

Your code should be commented in order to clarify implementation details, and desciption about the inputs and outputs of functions.

Note: There is no need to send the dataset, since I will download it and run your scripts directly. Therefore, make sure that you do not transform/change the raw data, and if there is some preprocessing involved, include it in your python file(s).


General Hints

    The tasks corresponding to these projects have been extensively studied. Don't hesitate to search online for more information (either tutorials or research papers, even pieces of code if you can adapt it).
    Some datasets are very large. Therefore, you don't need to use all the data, but instead you can extract a subset of it (for instance, only a few languages for language identification, only a few images/classes for image recognition etc.) to have a lighter dataset / model / training procedure.
    Some datasets are provided with a train / test split, but not always. Either way, you can create your own split (a good rule of thumb can be 80% training and 20 % testing).
    It is strongly advised to use validation in order to monitor training (see 'bonus work' in lab 2.2). You can use part of the training data (e.g., 10%) as a validation set.
    It is good practice to start with a light model (very few layers/parameters) and dataset (subset of your whole dataset) for prototyping. The performance won't be very good, but it's useful to check if there are any error in the train/test procedure. Once everything runs smoothly, you can increase the size of the model and use more data.
    If your project includes convolutional or recurrent neural networks, don't wait the corresponding lab: you can already start working on it (you can basically do everything, just using a 'dummy' MLP model instead of a CNN/RNN).

Modifié le: vendredi 21 octobre 2022, 12:46

In [ ]:
# STEP 1: Download the data and preprocess it as required for the given task (if needed).
#########


In [3]:
# STEP 2: Write a Dataset class for creating the train and test datasets (and corresponding dataloaders).
#########


In [4]:
# STEP 3: Define the neural network model.
#########


In [5]:
# STEP 4: Define the hyperparameters to create an instance of the model 
# (e.g., hidden space size, number of convolution kernels...) 
# as well as the parameters required to train neural network (e.g., learning rate).
#########


In [ ]:
# STEP 5: Write the training loop for training the model.
#########


In [ ]:
# STEP 6: Evaluate the model on the test data. 
# In this part, it is expected to choose an appropriate evaluation metric based on your task. 
# For instance, for classification task, accuracy should be computed (but you can also search for 'precision' and 'recall').
#########


In [6]:
# STEP 7: Save the trained model parameters, and the obtained results if needed.
#########
